In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/mushrooms.csv')

In [3]:
data.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [66]:
### Decision tree implement

import numpy as np
import pandas as pd
import math

class TreeNode:
    def __init__(self, data, output):
        # data represents the feature upon which the node was split
        # when  fitting the training data
        self.data = data
        # children of a node are stored as a dictionary with key being
        # the value of feature upon which the node was split
        # and the corresponding value stores the child TreeNode
        self.children = {}
        # output represents the class with current majority at this instance of the decision tree
        self.output = output
        # index will be used to assign a unique index to each node
        self.index = -1
    
    def add_child(self, feature_value, obj):
        self.children[feature_value] = obj


In [108]:
class DecisionTreeClassifier:
    def __init__(self):
        # root represents the root node of the decision tree built after
        # fitting the training data
        self.__root = None
        
    def __count_unique(self, Y):
        # returns a dictionary with keys as unique values of Y (i.e no of classes)
        # and the corresponding value as its frequency
        d = {}
        for i in Y:
            if i not in d:
                d[i] = 1
            else:
                d[i] += 1
        return d
    
    def __entropy(self, Y):
        # returns the entropy
        freq_map = self.__count_unique(Y)
        entropy_ = 0
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            entropy_ += (-p)*math.log2(p)
        return entropy_

    def __gain_ratio(self, X, Y, selected_feature):
        # return the gain ratio
        info_orig = self.__entropy(Y) # info_orig represents entropy before splitting
        info_f = 0 # info_f represents entropy after splitting upon the selected feature
        split_info = 0
        values = set(X[:, selected_feature])
        df = pd.DataFrame(X)
        # adding Y values as the last column in the dataframe
        df[df.shape[1]] = Y
        initial_size = df.shape[0]
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            info_f += (current_size/initial_size)*self.__entropy(df1[df1.shape[1]-1])
            split_info += (-current_size/initial_size)*math.log2(current_size/initial_size)
        
        # to handle the case when split info = 0 which leads to division by 0 error
        if split_info == 0:
            return math.inf
        
        info_gain = info_orig - info_f
        gain_ratio = info_gain / split_info
        return gain_ratio
    
    def __gini_index(self, Y):
        # returns the gini index
        freq_map = self.__count_unique(Y)
        gini_index_ = 1
        total = len(Y)
        for i in freq_map:
            p = freq_map[i]/total
            gini_index_ -= p**2
    
    def __gini_gain(self, X, Y, selected_feature):
        # returns the gini gain
        gini_orig = self.__gini_index(Y) # gini_orig represents gini index before splitting
        gini_split_f = 0 # gini_split_f represents gini index after splitting upon the selected feature
        values = set(X[:, selected_feature])
        df = pd.DataFrame(X)
        # Adding Y values as the last column in the dataframe
        df[df.shape[1]] = Y
        initial_size = df.shape[0]
        for i in values:
            df1 = df[df[selected_feature] == i]
            current_size = df1.shape[0]
            gini_split_f += (current_size/initial_size)*self.__gini_index(df[df1.shape[1]-1])
            
        gini_gain_ = gini_orig - gini_split_f
        return gini_gain_
    
    def __decision_tree(self, X, Y, features, level, metric, classes):
        # returns the root of the Decision Tree(which consists of TreeNodes) built after fitting the training data
        # here nodes are printed as in PREORDER traversal
        # classes represents the different classes present in the classification problem
        # metric can take value gain_ratio or gini_index
        # level represents depth of the tree
        # we split a node on a particular feature only once (in a given root to leaf node path)
        
        
        # if the node consists of only 1 class
        if len(set(Y)) == 1:
#             print("Level", level)
            output = None
            for i in classes:
                if i in Y:
                    output = i
#                     print("Count of",i,"=",len(Y))
                else:
                    continue
#                     print("Count of",i,"=",0)
#             if metric == "gain_ratio":
#                 print("Current Entropy is = 0.0")
#             else:
#                 print("Current Gini Index is = 0.0")
            
#             print("Reached leaf Node")
#             print()
            return TreeNode(None, output)
        
        # if we have run out of features to split upon
        # in this case we will output the class with maximum count
        if len(features) == 0:
#             print("Level", level)
            freq_map = self.__count_unique(Y)
            output = None
            max_count = -math.inf
            for i in classes:
                if i not in freq_map:
                    continue
#                     print("count of",i,"=",0)
                else:
                    if freq_map[i] > max_count:
                        output = i
                        max_count = freq_map[i]
#                     print("Count of",i,"=",freq_map[i])
                
#             if metric == "gain_ratio":
#                 print("Current Entropy is =",self.entropy_(Y))
#             elif metric == "gini_index":
#                 print("Current Gini Index is =", self.__gini_index(Y))
            
#             print("Reached leaf Node")
#             print()
            return TreeNode(None, output)
        
        
        # Finding the best feature to split upon
        max_gain = -math.inf
        final_feature = None
        for f in features:
            if metric == "gain_ratio":
                current_gain = self.__gain_ratio(X,Y,f)
            elif metric == "gini_index":
                current_gain = self.__gini_gain(X,Y,f)
            
            if current_gain > max_gain:
                max_gain = current_gain
                final_feature = f
        
#         print("Level", level)
        freq_map = self.__count_unique(Y)
        output = None
        max_count = -math.inf
        
        for i in classes:
            if i not in freq_map:
                continue
#                 print("Count of",i,"=",0)
            else:
                if freq_map[i] > max_count:
                    output = i
                    max_count = freq_map[i]
#                 print("Count of",i,"=",freq_map[i])
#         if metric == "gain_ratio":
#             print("Current Entropy is =",self.__entropy(Y))
#             print("Splitting on feature X[",final_feature,"] with gain ratio ", max_gain,sep="")
#             print()
#         elif metric == "gini_index":
#             print("Current Gini Index is =", self.__gini_index(Y))
#             print("Splitting on feature X[",final_feature,"] with gini gain ", max_gain,sep="")
#             print()
            
        
        unique_values = set(X[:, final_feature]) # unique_values represents the unique values of the feature selected
        df = pd.DataFrame(X)
        # adding Y values as the last column in the dataframe
        df[df.shape[1]] = Y
        
        current_node = TreeNode(final_feature, output)
        
        # now removing the selected feature from the list as we do not
        # do not want to split on one featuer more than once (in a given root to leaf node path)
        index = features.index(final_feature)
        features.remove(final_feature)
        for i in unique_values:
            # creating a new dataframe with value of selected feature = i
            df1= df[df[final_feature] == i]
            # segregating the X and Y values and recuresively calling on the splits
            node = self.__decision_tree(df1.iloc[:,0:df1.shape[1]-1].values,df1.iloc[:, df1.shape[1]-1].values, features,level+1, metric, classes)
            current_node.add_child(i, node)
            
        # add the removed feature
        features.insert(index, final_feature)
            
        return current_node
    
    def fit(self, X, Y, metric="gain_ratio"):
        # fits to the given training data
        # metric can take value gain_ratio or gini_index
        features = [i for i in range(len(X[0]))]
        classes = set(Y)
        level = 0
        if metric != "gain_ratio":
            if metric != "gini_index":
                metric = "gain_ratio" # if user entered a value which was neither gini_index nor gain_ratio
        self.__root = self.__decision_tree(X,Y,features, level, metric, classes)
    
    def __predict_for(self, data, node):
        # predicts the class for a given testing point and returns the answer
        
        # we have reached a leaf node
        if len(node.children) == 0:
            return node.output
        
        val = data[node.data] # represents the value of feature on which the split was made
        if val not in node.children:
            return node.output
        
        # recursively call on the splits
        return self.__predict_for(data, node.children[val])
    
    def predict(self, X):
        # this function returns Y predicted
        # X should be a 2-D np array
        Y = np.array([0 for i in range(len(X))])
        for i in range(len(X)):
            Y[i] = self.__predict_for(X[i], self.__root)
        return Y
    
    def score(self, X, Y):
        # return the mean accuracy
        Y_pred = self.predict(X)
        count = 0
        for i in range(len(Y_pred)):
            if Y_pred[i] == Y[i]:
                count += 1
        return count/len(Y_pred)


In [83]:
clf1 = DecisionTreeClassifier()

x = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

y = np.array([0,
              1,
              1,
              1]) 
clf1.fit(x,y)
Y_pred = clf1.predict(x)
print("Predictions :",Y_pred)
print()
print("Score :",clf1.score(x,y)) # Score on training data

0
1
2
3
Predictions : [0 1 1 1]

0
1
2
3
Score : 1.0


In [101]:
X


array([[5, 2, 4, ..., 2, 3, 5],
       [5, 2, 9, ..., 3, 2, 1],
       [0, 2, 8, ..., 3, 2, 3],
       ...,
       [2, 2, 4, ..., 0, 1, 2],
       [3, 3, 4, ..., 7, 4, 2],
       [5, 2, 4, ..., 4, 1, 2]])

In [84]:
# apply decision tree for our data
# data purification
data = pd.read_csv('data/mushrooms.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
class                       8124 non-null object
cap-shape                   8124 non-null object
cap-surface                 8124 non-null object
cap-color                   8124 non-null object
bruises                     8124 non-null object
odor                        8124 non-null object
gill-attachment             8124 non-null object
gill-spacing                8124 non-null object
gill-size                   8124 non-null object
gill-color                  8124 non-null object
stalk-shape                 8124 non-null object
stalk-root                  8124 non-null object
stalk-surface-above-ring    8124 non-null object
stalk-surface-below-ring    8124 non-null object
stalk-color-above-ring      8124 non-null object
stalk-color-below-ring      8124 non-null object
veil-type                   8124 non-null object
veil-color                  8124 non-null object
ring-number

In [85]:
# In this data We can find some null data like '?'
#'veil-type' and 'stalk-root' has Some NaN values 
#so Delete these 2 columns from the data
data.drop('veil-type', axis=1, inplace=True)
data.drop('stalk-root', axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 21 columns):
class                       8124 non-null object
cap-shape                   8124 non-null object
cap-surface                 8124 non-null object
cap-color                   8124 non-null object
bruises                     8124 non-null object
odor                        8124 non-null object
gill-attachment             8124 non-null object
gill-spacing                8124 non-null object
gill-size                   8124 non-null object
gill-color                  8124 non-null object
stalk-shape                 8124 non-null object
stalk-surface-above-ring    8124 non-null object
stalk-surface-below-ring    8124 non-null object
stalk-color-above-ring      8124 non-null object
stalk-color-below-ring      8124 non-null object
veil-color                  8124 non-null object
ring-number                 8124 non-null object
ring-type                   8124 non-null object
spore-print

In [86]:
data['class'].unique()

array(['p', 'e'], dtype=object)

In [87]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in data.columns:
    data[i] = le.fit_transform(data[i])
data.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1


In [88]:
Y = data['class']
data1 = data
data1.drop('class', axis=1,inplace=True)

In [89]:
data1.head(5)

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5,2,4,1,6,1,0,1,4,0,2,2,7,7,2,1,4,2,3,5
1,5,2,9,1,0,1,0,0,4,0,2,2,7,7,2,1,4,3,2,1
2,0,2,8,1,3,1,0,0,5,0,2,2,7,7,2,1,4,3,2,3
3,5,3,8,1,6,1,0,1,5,0,2,2,7,7,2,1,4,2,3,5
4,5,2,3,0,5,1,1,0,4,1,2,2,7,7,2,1,0,3,0,1


In [104]:
X = data1
X.head(5)
X = X.values.tolist()
Y = Y.values.tolist()
# Y = pd.DataFrame(Y).to_numpy()
X = np.asarray(X)
Y = np.asarray(Y)
Y

array([1, 0, 0, ..., 0, 1, 0])

In [105]:
# splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3)

In [109]:
clf1 = DecisionTreeClassifier()
clf1.fit(X_train, y_train)



In [110]:
print("Score :",clf1.score(X_test,y_test))

Score : 1.0


In [112]:
# using the inbuilt sklearn Decision tree
import sklearn.tree
clf2 = sklearn.tree.DecisionTreeClassifier()
clf2.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [113]:
sklearn_score = clf2.score(X_test, y_test)
print("Score: ", sklearn_score)

Score:  1.0
